In [2]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


data = pd.read_csv("sqli.csv")  
data.columns = ['Query', 'Label']


sqli_patterns = [
    r"'.+?(--|#|\/\*|\|)",
    r"\b(union|select).+?(from|where).+?\d+=\d+",
    r"\b(drop|alter|truncate)\s+(table|database)",
    r";\s*(declare|exec|shutdown)",
    r"\b(and|or)\s+[\w\d]+\s*=\s*[\w\d]+\s*(--|#)",
    r"'.*?(sleep|benchmark|waitfor|pg_sleep)\(",
    r"\{\s*\"\$where\"\s*:\s*\".+?\b(OR|AND)\b.+?\"\s*\}",
    r"\bEXEC\b.*?\(",
    r"\bLOAD_FILE\s*\(.*?\)",
    r"\bxp_cmdshell\b"
]


def preprocess_query(query):
    query = query.lower()
    if any(ptrn in query for ptrn in ["--", "#", "/*", "*/", "' or ", "' and "]):
        query = re.sub(r"('[^']*')", lambda m: m.group(1).replace(' ', '_'), query)
    else:
        query = re.sub(r"'[^']*'", 'STR', query)
    query = re.sub(r'\b\d+\b', 'NUM', query)
    query = re.sub(r'[^\w\s=/*#\-<>+!@]', ' ', query)
    return ' '.join(query.split())


data['clean_query'] = data['Query'].apply(preprocess_query)


x = data['clean_query']
y = data['Label']


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(
    ngram_range=(1, 3),
    analyzer='word',
    max_features=10000
)
x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec = vectorizer.transform(x_test)


smote = SMOTE(random_state=42)
x_train_balanced, y_train_balanced = smote.fit_resample(x_train_vec, y_train)


model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(x_train_balanced, y_train_balanced)


y_pred = model.predict(x_test_vec)
print(classification_report(y_test, y_pred))


def predict_query(query):
 
    if any(re.search(pattern, query, re.I) for pattern in sqli_patterns):
        return 1
    

    clean = preprocess_query(query)
    vec = vectorizer.transform([clean])
    return model.predict(vec)[0]

import pickle


with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

test_queries = [
    "SELECT * FROM products",
    "admin' OR 1=1 --",
    "UPDATE users SET last_login=NOW()",
    "' UNION SELECT * FROM users --"
]

for query in test_queries:
    print(f"Query: {query[:50]}... -> Prediction: {predict_query(query)}")

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3893
           1       1.00      0.90      0.95      2291

    accuracy                           0.96      6184
   macro avg       0.97      0.95      0.96      6184
weighted avg       0.96      0.96      0.96      6184

Query: SELECT * FROM products... -> Prediction: 0
Query: admin' OR 1=1 --... -> Prediction: 1
Query: UPDATE users SET last_login=NOW()... -> Prediction: 0
Query: ' UNION SELECT * FROM users --... -> Prediction: 1


In [4]:
test_queries = [
    "SELECT * FROM products WHERE name LIKE '%admin%'",
    "UPDATE users SET last_login = NOW() WHERE username = 'admin'",
    "'John O'Connor",
    "' OR 1=1 --",
    "admin'--",
    "'; DROP TABLE users --",
    "' AND (SELECT 1 FROM (SELECT BENCHMARK(1e6,MD5(NOW())))a) --",
    "'; DECLARE @q VARCHAR(100);SET @q='SELECT * FROM users'; EXEC(@q);--",
    "' UNION SELECT column_name, NULL FROM INFORMATION_SCHEMA.columns WHERE table_name = 'users' #",
    "hello wrld"
]

print("\nTest Query Predictions:")
for q in test_queries:
    print(f"Query: {q[:60]}... -> Prediction: {predict_query(q)}")


Test Query Predictions:
Query: SELECT * FROM products WHERE name LIKE '%admin%'... -> Prediction: 0
Query: UPDATE users SET last_login = NOW() WHERE username = 'admin'... -> Prediction: 0
Query: 'John O'Connor... -> Prediction: 0
Query: ' OR 1=1 --... -> Prediction: 1
Query: admin'--... -> Prediction: 1
Query: '; DROP TABLE users --... -> Prediction: 1
Query: ' AND (SELECT 1 FROM (SELECT BENCHMARK(1e6,MD5(NOW())))a) --... -> Prediction: 1
Query: '; DECLARE @q VARCHAR(100);SET @q='SELECT * FROM users'; EXE... -> Prediction: 1
Query: ' UNION SELECT column_name, NULL FROM INFORMATION_SCHEMA.col... -> Prediction: 1
Query: hello wrld... -> Prediction: 0
